In [1]:
import pysam
from pysam import VariantFile as vcf
import operator
from math import log2
import pandas as pd
from pandas import DataFrame as dataframe
import matplotlib.pyplot as plt
import numpy as np

from scipy.spatial.distance import pdist, squareform
import scipy
import  os
import os.path
import matplotlib.colors as mcolors
from scipy import stats
import csv
import requests
import json
from xml.etree import ElementTree as ET
from statsmodels.stats.multitest import multipletests
beitiaoguode=[]
df_csv2=pd.read_csv("/data2/wangxuedong/mhc_test_data/first_review/find_region/dis_csv/tables3_01part.csv").values.tolist()


all_mean_arr=0.03411387460093794
all_std_arr=0.01334116014092687
mean_dict_different_race={'ACB': 0.046621686769679045,
 'ASW': 0.0465892685112964,
 'BEB': 0.035970363185117774,
 'CDX': 0.03266259792789275,
 'CEU': 0.03488434818325704,
 'CHB': 0.03285897897059889,
 'CHS': 0.032796236687928325,
 'CLM': 0.036503986698316906,
 'ESN': 0.04643492431530403,
 'FIN': 0.034935842703769975,
 'GBR': 0.03498406394197642,
 'GIH': 0.035593525252319486,
 'GWD': 0.045836194805001416,
 'IBS': 0.035140910353782764,
 'ITU': 0.0347164844670136,
 'JPT': 0.03273063391657775,
 'KHV': 0.033023382087129426,
 'LWK': 0.0466350005070252,
 'MSL': 0.047257776857587634,
 'MXL': 0.03529323182476227,
 'PEL': 0.03222809053641309,
 'PJL': 0.03450825905794522,
 'PUR': 0.037936602232609126,
 'STU': 0.03454596363552346,
 'TSI': 0.03508890911947475,
 'YRI': 0.04649868546958651}
std_dict_different_race={'ACB': 0.01660837078934563,
 'ASW': 0.016852430054156438,
 'BEB': 0.016339814570213686,
 'CDX': 0.01657929168371163,
 'CEU': 0.01615285680476383,
 'CHB': 0.0167769938917591,
 'CHS': 0.016361466363591634,
 'CLM': 0.016076712212741314,
 'ESN': 0.016439364809682746,
 'FIN': 0.016395558138330903,
 'GBR': 0.01654186443581865,
 'GIH': 0.016487954511321803,
 'GWD': 0.01637735033664222,
 'IBS': 0.01625740426618882,
 'ITU': 0.015999114837619923,
 'JPT': 0.016777856667394144,
 'KHV': 0.016805860157616894,
 'LWK': 0.01656542896110204,
 'MSL': 0.0168657687428544,
 'MXL': 0.01631757749279766,
 'PEL': 0.015903810806753623,
 'PJL': 0.015785059411911464,
 'PUR': 0.016016827217755162,
 'STU': 0.01615897181252148,
 'TSI': 0.016499539576161064,
 'YRI': 0.01656520483641468}

allrecords=[]
#in csv2
for record in df_csv2:
    
    if type(record[6])!=float:
     
        if record[1]=="All":
            if record[2]==0:#no mhc csvfiles                        
                position_of_gene=record[6]
                chr=position_of_gene.split(":")[0]
                start_position=int(position_of_gene.split(":")[1].split("-")[0])

                end_position=int(position_of_gene.split(":")[1].split("-")[1])
                # filenames=os.listdir("/data2/wangxuedong/mhc_test_data/first_review/csvfiles_integration/")
                # for filename in filenames:
                    #if filename.startswith("pos_prob_split_"+chr+"_"):
                df_split=pd.read_csv("/data2/wangxuedong/mhc_test_data/first_review/csvfiles_integration/"+chr+".csv")  

                allrigion_P_top_1000=df_split[(df_split["Unnamed: 0"]>=start_position) & (df_split["Unnamed: 0"]<=end_position)]["P"].to_list()
                if allrigion_P_top_1000:
                    mean_100_P=np.mean(allrigion_P_top_1000)
                    std_100_P=np.std(allrigion_P_top_1000, ddof=1)
                    pvalue=round((1-stats.norm.cdf(mean_100_P, all_mean_arr, all_std_arr)),15)
                    onerecord=[]
                    onerecord+=record[1:9]
                    onerecord+=[mean_100_P,std_100_P,pvalue]
                    allrecords.append(onerecord)
                        

                else:
                    print("continue",record)
                    beitiaoguode.append(record)
                    continue

            else:
                position_of_gene=record[6]
                start_position=int(position_of_gene.split(":")[1].split("-")[0])
                end_position=int(position_of_gene.split(":")[1].split("-")[1])
                df_mhc=pd.read_csv("/data2/wangxuedong/mhc_test_data/first_review/csvfiles/mhc.csv")
                allrigion_P_top_1000=df_mhc[(df_mhc["Unnamed: 0"]>=start_position) & (df_mhc["Unnamed: 0"]<=end_position)]["P"].to_list()
                if allrigion_P_top_1000:
                    mean_100_P=np.mean(allrigion_P_top_1000)
                    std_100_P=np.std(allrigion_P_top_1000, ddof=1)
                    pvalue=round((1-stats.norm.cdf(mean_100_P, all_mean_arr, all_std_arr)),15)
                    onerecord=[]
                    onerecord+=record[1:9]
                    onerecord+=[mean_100_P,std_100_P,pvalue]
                    allrecords.append(onerecord)
                else:
                    print("continue",record)
                    beitiaoguode.append(record)
                    continue

        else:#race!=all
            if record[2]==0:#no mhc
                position_of_gene=record[6]
                chr=position_of_gene.split(":")[0]
                start_position=int(position_of_gene.split(":")[1].split("-")[0])

                end_position=int(position_of_gene.split(":")[1].split("-")[1])
                # filenames=os.listdir("/data2/wangxuedong/mhc_test_data/first_review/race_integration/")
                # for filename in filenames:
                    # if filename.startswith("split_"+chr+"_"):
                df_split=pd.read_csv("/data2/wangxuedong/mhc_test_data/first_review/race_integration/"+chr+".csv")  
            
                allrigion_P_top_1000=df_split[(df_split["Unnamed: 0"]>=start_position) & (df_split["Unnamed: 0"]<=end_position)][record[1]].to_list()
                if allrigion_P_top_1000:
                    mean_100_P=np.mean(allrigion_P_top_1000)
                    std_100_P=np.std(allrigion_P_top_1000, ddof=1)
                    pvalue=round((1-stats.norm.cdf(mean_100_P,mean_dict_different_race[record[1]],std_dict_different_race[record[1]])),15)
                        
                    onerecord=[]
                    onerecord+=record[1:9]
                    onerecord+=[mean_100_P,std_100_P,pvalue]
                    allrecords.append(onerecord)
                else:
                    print("continue",record)
                    beitiaoguode.append(record)
                    continue
                    # else:
                    #     continue
            else:#ismhc different race
                position_of_gene=record[6]
                start_position=int(position_of_gene.split(":")[1].split("-")[0])
                end_position=int(position_of_gene.split(":")[1].split("-")[1])
                df_mhc=pd.read_csv("/data2/wangxuedong/mhc_test_data/first_review/race/posmhc.csv")
                
                allrigion_P_top_1000=df_mhc[(df_mhc["Unnamed: 0"]>=start_position) & (df_mhc["Unnamed: 0"]<=end_position)][record[1]].to_list()
                if allrigion_P_top_1000:                
                    mean_100_P=np.mean(allrigion_P_top_1000)
                    std_100_P=np.std(allrigion_P_top_1000, ddof=1)
                    pvalue=round((1-stats.norm.cdf(mean_100_P,mean_dict_different_race[record[1]],std_dict_different_race[record[1]])),15)              
                    onerecord=[]
                    onerecord+=record[1:9]
                    onerecord+=[mean_100_P,std_100_P,pvalue]
                    allrecords.append(onerecord)
                else:
                    print("continue",record)
                    beitiaoguode.append(record)
                    continue
    else:
        print("float number")

        onerecord=[]
        onerecord+=record[1:9]
        onerecord+=["null","null","null"]
        allrecords.append(onerecord)

#generatedf
def generatedf(columns,allrecords):
    dictforDF=dict()
    for i in range(len(columns)):
        midarr=[]
        for record in allrecords:           
            midarr.append(record[i])
        dictforDF[columns[i]]=midarr
    dataframe(dictforDF).to_csv("tables3_part2_nocorr.csv")
    return dataframe(dictforDF)


dfnew=generatedf(columns=["Race","isMHC","Position of disassortative mating region","Gene ID","Gene name","Position of gene","is_complement","Gene description","Average of P","Std of P","P-value"],allrecords=allrecords)


pvalue_list=dfnew["P-value"].tolist()
corr_pvals = multipletests(pvalue_list, method='fdr_bh')[1]
dfnew = dfnew.assign(corr_pvals=corr_pvals)
dfnew.to_csv("tables3_part2_withcorr.csv")

continue [82, 'CEU', 0, 'chr17:45825257-46352295', 'ENSG00000238723', 'Y_RNA', 'chr17:46215070-46215171', 'R', 'Y RNA [Source:RFAM;Acc:RF00019]']
continue [89, 'CEU', 0, 'chr17:45825257-46352295', 'ENSG00000265411', 'RN7SL656P', 'chr17:46319679-46319959', 'F', 'RNA, 7SL, cytoplasmic 656, pseudogene [Source:HGNC Symbol;Acc:HGNC:46672]']
continue [117, 'CLM', 0, 'chr17:45825257-46352295', 'ENSG00000238723', 'Y_RNA', 'chr17:46215070-46215171', 'R', 'Y RNA [Source:RFAM;Acc:RF00019]']
continue [124, 'CLM', 0, 'chr17:45825257-46352295', 'ENSG00000265411', 'RN7SL656P', 'chr17:46319679-46319959', 'F', 'RNA, 7SL, cytoplasmic 656, pseudogene [Source:HGNC Symbol;Acc:HGNC:46672]']
continue [171, 'GBR', 0, 'chr17:45825257-46352295', 'ENSG00000238723', 'Y_RNA', 'chr17:46215070-46215171', 'R', 'Y RNA [Source:RFAM;Acc:RF00019]']
continue [178, 'GBR', 0, 'chr17:45825257-46352295', 'ENSG00000265411', 'RN7SL656P', 'chr17:46319679-46319959', 'F', 'RNA, 7SL, cytoplasmic 656, pseudogene [Source:HGNC Symbol;

/home/wangxuedong/app/miniconda3/envs/mhc/lib/python3.8/site-packages/numpy/core/_methods.py:265: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/wangxuedong/app/miniconda3/envs/mhc/lib/python3.8/site-packages/numpy/core/_methods.py:257: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


continue [343, 'PUR', 0, 'chr17:45825257-46352295', 'ENSG00000238723', 'Y_RNA', 'chr17:46215070-46215171', 'R', 'Y RNA [Source:RFAM;Acc:RF00019]']
continue [350, 'PUR', 0, 'chr17:45825257-46352295', 'ENSG00000265411', 'RN7SL656P', 'chr17:46319679-46319959', 'F', 'RNA, 7SL, cytoplasmic 656, pseudogene [Source:HGNC Symbol;Acc:HGNC:46672]']
continue [388, 'TSI', 0, 'chr17:45825257-46352295', 'ENSG00000238723', 'Y_RNA', 'chr17:46215070-46215171', 'R', 'Y RNA [Source:RFAM;Acc:RF00019]']
continue [395, 'TSI', 0, 'chr17:45825257-46352295', 'ENSG00000265411', 'RN7SL656P', 'chr17:46319679-46319959', 'F', 'RNA, 7SL, cytoplasmic 656, pseudogene [Source:HGNC Symbol;Acc:HGNC:46672]']
continue [434, 'ACB', 1, 'chr6:29720403-30011739', 'ENSG00000270896', nan, 'chr6:29763258-29763392', 'R', 'mitochondrial coiled-coil domain 1 (MCCD1) pseudogene']


/home/wangxuedong/app/miniconda3/envs/mhc/lib/python3.8/site-packages/numpy/core/_methods.py:265: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/wangxuedong/app/miniconda3/envs/mhc/lib/python3.8/site-packages/numpy/core/_methods.py:257: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


continue [489, 'ACB', 1, 'chr6:30994370-31528792', 'ENSG00000277402', 'MIR6891', 'chr6:31355224-31355316', 'R', 'microRNA 6891 [Source:HGNC Symbol;Acc:HGNC:50243]']
continue [549, 'All', 1, 'chr6:29720403-29913914', 'ENSG00000270896', nan, 'chr6:29763258-29763392', 'R', 'mitochondrial coiled-coil domain 1 (MCCD1) pseudogene']


...

continue [3149, 'TSI', 1, 'chr6:30994370-31528792', 'ENSG00000277402', 'MIR6891', 'chr6:31355224-31355316', 'R', 'microRNA 6891 [Source:HGNC Symbol;Acc:HGNC:50243]']
continue [3209, 'YRI', 1, 'chr6:29720403-30120966', 'ENSG00000270896', nan, 'chr6:29763258-29763392', 'R', 'mitochondrial coiled-coil domain 1 (MCCD1) pseudogene']


/home/wangxuedong/app/miniconda3/envs/mhc/lib/python3.8/site-packages/numpy/core/_methods.py:265: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/wangxuedong/app/miniconda3/envs/mhc/lib/python3.8/site-packages/numpy/core/_methods.py:257: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


continue [3242, 'YRI', 1, 'chr6:29720403-30120966', 'ENSG00000275856', nan, 'chr6:30058115-30058190', 'F', nan]


/home/wangxuedong/app/miniconda3/envs/mhc/lib/python3.8/site-packages/numpy/core/_methods.py:265: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/wangxuedong/app/miniconda3/envs/mhc/lib/python3.8/site-packages/numpy/core/_methods.py:257: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


continue [3274, 'YRI', 1, 'chr6:30994370-31528792', 'ENSG00000277402', 'MIR6891', 'chr6:31355224-31355316', 'R', 'microRNA 6891 [Source:HGNC Symbol;Acc:HGNC:50243]']


In [2]:
beitiaoguode=[tuple(b) for b in beitiaoguode]
norepeat=list(set(beitiaoguode))
len(norepeat)

75

In [3]:
norepeat

[(684,
  'ASW',
  1,
  'chr6:29720403-30120966',
  'ENSG00000275856',
  nan,
  'chr6:30058115-30058190',
  'F',
  nan),
 (1206,
  'CHS',
  1,
  'chr6:30994370-31528792',
  'ENSG00000277402',
  'MIR6891',
  'chr6:31355224-31355316',
  'R',
  'microRNA 6891 [Source:HGNC Symbol;Acc:HGNC:50243]'),
 (395,
  'TSI',
  0,
  'chr17:45825257-46352295',
  'ENSG00000265411',
  'RN7SL656P',
  'chr17:46319679-46319959',
  'F',
  'RNA, 7SL, cytoplasmic 656, pseudogene [Source:HGNC Symbol;Acc:HGNC:46672]'),
 (2434,
  'MSL',
  1,
  'chr6:29720403-30011739',
  'ENSG00000270896',
  nan,
  'chr6:29763258-29763392',
  'R',
  'mitochondrial coiled-coil domain 1 (MCCD1) pseudogene'),
 (178,
  'GBR',
  0,
  'chr17:45825257-46352295',
  'ENSG00000265411',
  'RN7SL656P',
  'chr17:46319679-46319959',
  'F',
  'RNA, 7SL, cytoplasmic 656, pseudogene [Source:HGNC Symbol;Acc:HGNC:46672]'),
 (343,
  'PUR',
  0,
  'chr17:45825257-46352295',
  'ENSG00000238723',
  'Y_RNA',
  'chr17:46215070-46215171',
  'R',
  'Y RNA [